In [ ]:
#Sina Gholami
#MIRCV 2021
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#To support GPU
!pip install opencv-python==4.4.0.46

In [ ]:
import os
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

# for showing images in the cell outputs (Jupyter Notebooks / Google Colab)
from IPython.display import display
from ipywidgets import Image


BASE_DIR = '/content/gdrive/My Drive/mircv2021'
DEEP_PROTO = BASE_DIR + '/data/caffe/train_val.prototxt'
DEEP_MODEL = BASE_DIR + '/data/caffe/bvlc_reference_caffenet.caffemodel'
SRC_FOLDER = BASE_DIR + '/data/coco_img'

DEEP_LAYER = 'relu7'
SIZE = (227, 227)
MEAN_VALUES = (104, 117, 123)  # BGR

K = 10

In [ ]:
filename="000000000724.jpg"
image1 = join(SRC_FOLDER, filename)
display(Image.from_file(image1, width=300, height=400))

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00H\x00H\x00\x00\xff\xe2\x0cXICC_PROFILE\x00\x01\x…

In [ ]:
img1 = cv2.imread(image1)
net = cv2.dnn.readNetFromCaffe(DEEP_PROTO, DEEP_MODEL)
blob1 = cv2.dnn.blobFromImage(img1, 1.0, SIZE, MEAN_VALUES, swapRB=False, crop=False)
net.setInput(blob1)
prob1= net.forward(DEEP_LAYER)
print(prob1)
print(prob1.shape)

prob1=prob1.flatten()
print(prob1)
print(prob1.shape)

[[-0.        -0.        -0.        ...  0.9809191 -0.        -0.       ]]
(1, 4096)
[-0.        -0.        -0.        ...  0.9809191 -0.        -0.       ]
(4096,)


In [ ]:
prob1 = prob1/np.linalg.norm(prob1)
print(prob1)


[-0.         -0.         -0.         ...  0.01489693 -0.
 -0.        ]


In [ ]:
s = np.dot(prob1, prob1)
print(s)

0.9999998


In [ ]:
filename = "000000001268.jpg"
image2 = join(SRC_FOLDER, filename)
display(Image.from_file(image2, width=300, height=400))

img2 = cv2.imread(image2)

blob2 = cv2.dnn.blobFromImage(img2, 1.0, SIZE, MEAN_VALUES, swapRB=False, crop=False)
net.setInput(blob2)
prob2= net.forward(DEEP_LAYER)

prob2=prob2.flatten()
prob2 = prob2/np.linalg.norm(prob2)

s = np.dot(prob1, prob2)
print(s)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00H\x00H\x00\x00\xff\xe2\x0cXICC_PROFILE\x00\x01\x…

0.2460078


In [ ]:
class DNNExtractor:    
    
    def __init__(self, net_proto_path, trained_model_path, size, mean_values=None):
        self.size = size
        self.mean_values = mean_values
        #TODO
        #Load Caffe Weights
        self.net = cv2.dnn.readNetFromCaffe(DEEP_PROTO, DEEP_MODEL)
        # to enable GPU (this won't work on Colab without recompiling opencv)
        #self.net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
        #self.net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)
    
    def extract(self, img_file, layer, normalize=False):
        #TODO
        #Load img_file and extract fetures calling DNN 
        img = cv2.imread(join(SRC_FOLDER, img_file))
        blob = cv2.dnn.blobFromImage(img, 1.0, self.size, self.mean_values, swapRB=False, crop=False)
        self.net.setInput(blob)
        prob = self.net.forward(layer)
        prob = prob.flatten()
        #normalize fetaures if normalize = True
        #return the features
        return prob/np.linalg.norm(prob) if normalize else prob
    
# it creates an instance of the DNNExtractor class
dnn = DNNExtractor(DEEP_PROTO, DEEP_MODEL, SIZE, MEAN_VALUES)

In [ ]:
def extract_features(img_folder):
  
    #TODO
    #get the list of image filenames

    filenames = [f for f in listdir(img_folder) if isfile(join(img_folder, f))]

    descriptors = []
    #TODO
    #cycle the img_folder directory to get the image files
    #extract the image features
    #add the features to the descriptors list
    count = 1
    for img in filenames[:1000]:
        descriptors.append(dnn.extract(img_file=img, layer=DEEP_LAYER, normalize=True))
        print(count, ") file name: " + img + " has finished!")
        count += 1
    #convert descriptors and filenames as Numpy arrays
    return np.array(descriptors), np.array(filenames)

descs, ids = extract_features(SRC_FOLDER)

In [ ]:
x = np.arange(9.).reshape(3, 3)
np.where(x>5, 1, 0)

In [ ]:
def search(queryF, k):
    #TODO
    #evaluate the dot products between descs and queryF
    #To sort the results create a zip between dot products and ids
    #then call the sorted function to sort them
    #return the k best results
    return sorted(zip(np.dot(descs, queryF), ids), reverse=True)[:k]

  
def display_results(results):
    for score, filename in results:
        print('{} - {:.3f}'.format(filename, score))  # :.3f = 3-decimal float

        filepath = join(SRC_FOLDER, filename)
        image = Image.from_file(filepath, width=300, height=400)
        display(image)

img_query = join(SRC_FOLDER, "000000321557.jpg")
query_features = dnn.extract(img_query, DEEP_LAYER, True)

display(Image.from_file(img_query, width=300, height=400))

res = search(query_features, K)
print(res)
display_results(res)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00H\x00H\x00\x00\xff\xdb\x00C\x00\x01\x01\x01\x01\…

[(1.0, '000000321557.jpg'), (0.52089334, '000000297147.jpg'), (0.5062005, '000000408830.jpg'), (0.4655091, '000000291791.jpg'), (0.45670915, '000000293245.jpg'), (0.455648, '000000455716.jpg'), (0.41400224, '000000144300.jpg'), (0.3783118, '000000190756.jpg'), (0.34579584, '000000291634.jpg'), (0.34548634, '000000061108.jpg')]
000000321557.jpg - 1.000


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00H\x00H\x00\x00\xff\xdb\x00C\x00\x01\x01\x01\x01\…

000000297147.jpg - 0.521


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00H\x00H\x00\x00\xff\xdb\x00C\x00\x01\x01\x01\x01\…

...

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00H\x00H\x00\x00\xff\xe2\x0cXICC_PROFILE\x00\x01\x…

000000291634.jpg - 0.346


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00H\x00H\x00\x00\xff\xe2\x0cXICC_PROFILE\x00\x01\x…

000000061108.jpg - 0.345


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00H\x00H\x00\x00\xff\xfe\x00\x0cAppleMark\n\xff\xe…

In [ ]:
#Optional task

#TODO
# binary descriptors. For each value set 1 if value > 0, 0 otherwise
descs_bin = np.where(descs > 0, 1, 0)

def search_bin(queryF_bin, k):
    hamming = []
    for v in descs_bin:
        hamming.append(np.sum(np.bitwise_xor(v, queryF_bin)))
    #TODO
    #evaluate Hamming distance between descs_bin and queryF_bin
    #then call the sorted function to sort them
    res = sorted(zip(hamming, ids[:1000]))[:k]
    #return the k best results
    return res

In [ ]:
#Optional task

#TODO
# binary query. For each value set 1 if value > 0, 0 otherwise
queryF_bin = np.where(query_features > 0, 1, 0)
res = search_bin(queryF_bin, K)
print(res)
display_results(res)


[(0, '000000321557.jpg'), (638, '000000408830.jpg'), (645, '000000144300.jpg'), (682, '000000297147.jpg'), (682, '000000455716.jpg'), (704, '000000190756.jpg'), (750, '000000256192.jpg'), (762, '000000061108.jpg'), (764, '000000102331.jpg'), (774, '000000491090.jpg')]
000000321557.jpg - 0.000


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00H\x00H\x00\x00\xff\xdb\x00C\x00\x01\x01\x01\x01\…

000000408830.jpg - 638.000


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00H\x00H\x00\x00\xff\xdb\x00C\x00\x01\x01\x01\x01\…

000000144300.jpg - 645.000


...

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00H\x00H\x00\x00\xff\xfe\x00\x0cAppleMark\n\xff\xe…

000000102331.jpg - 764.000


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00H\x00H\x00\x00\xff\xe2\x0cXICC_PROFILE\x00\x01\x…

000000491090.jpg - 774.000


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00H\x00H\x00\x00\xff\xfe\x00\x0cAppleMark\n\xff\xe…